## Feature Engineering and CV based Winners' Solutions

Try to use MAP@7 as `feval` for xgboost, failed because no way to tell xgboost how to group results by users. **Can only do this after train is finished.**

New in this notebook:
- A hacky implementation of MAP@7 evaluation. 
- This method is suitable when training on one month and validate on another month, since ncodpers is the key in ground truth dictionaries.
- This method **only works if the MAP functions and training codes are in the same notebook**.
- The MAP values shown during training is averaged by the number of customers with new products, so very different from LB. But it gives a sense of if the algorithm is good or not.

To-do: 
- mean encoding of products grouped by combinations of: canal_entrada, segmento, cod_prov
- Time since change and lags for a few non-product features: 
    - segmento
    - ind_actividad_cliente
    - cod_prov
    - canal_entrada
    - indrel_1mes
    - tiprel_1mes


Features:
- before eda_4_29
    - average of products for each (customer, product) pair
    - exponent weighted average of products each (customer, product) pair
    - time since presence of products, distance to the first 1
    - time to the last positive flank (01)
    - time to the last negative flank (10)
    - time to the last 1, to the nearest product purchase
    - time to the first 1, to the first product purchase
    - Trained@2015-06-28, validated@2015-12-28, mlogloss=1.28481
    - Private score: 0.0302054, public score: 0.0298683
- before eda_4_25
    - customer info in the second month
    - products in the first month
    - combination of first and second month `ind_actividad_cliente`
    - combination of first and second month `tiprel_1mes`
    - combination of first month product by using binary number (`target_combine`)
    - encoding `target_combine` with 
        - mean number of new products
        - mean number of customers with new products
        - mean number of customers with each new products
    - Count patterns in the last `max_lag` months
    - Number of month to the last time the customer purchase each product
        - CV@2015-12-28: mlogloss=1.29349
        - Private score: 0.0302475, public score: 0.0299266
- eda_4_25
    - Use all available history data
        - E.g., for 2016-05-28 train data, use all previous months, for 2015-02-28, use 1 lag month. 
        - Need to create test set that use the same amount of previous months for each training data set. 
        - This is from [the second winner's solution](https://www.kaggle.com/c/santander-product-recommendation/discussion/26824), his bold part in paragraph 4.
    - Combine models trained on 2016-05-28 and 2015-06-28:
        - Private score: 0.0304583, public score: 0.0300839
        - This is to catch both seasonality and trend, presented in 2015-06-28 and 2016-05-28, respectively. 
        - This idea is mentioned by many winners, like [11-th winner](https://www.kaggle.com/c/santander-product-recommendation/discussion/26823) and [14-th winner](https://www.kaggle.com/c/santander-product-recommendation/discussion/26808)

- eda_4_27
    - put 2015-06-28 and 2016-05-28 in the same data set, with the same lag=5
        - Private score:0.0303096, public score: 0.0299867
        - Different as [11-th winner's discussion](https://www.kaggle.com/c/santander-product-recommendation/discussion/26823)
            > We tested this by adding 50% of May-16 data to our June model and sure enough, we went from 0.0301 to 0.0303. Then, we built separate models for Jun and May, but the ensemble didn’t work. We weren’t surprised because June data is better for seasonal products, and May data is better for trend products. And vice-versa, June data is bad for trend products and May data is bad for seasonal products. So, they sort of cancelled each other out.

        - But my score is always worse than theirs, maybe this is the reason why we have different observations

# Compare two weights

In [1]:
from santander_helper import *

In [ ]:
param = {'objective': 'multi:softprob', 
         'eta': 0.05, 
         'max_depth': 8, 
         'silent': 1, 
         'num_class': len(target_cols),
         'eval_metric': 'mlogloss',
         'min_child_weight': 1,
         'subsample': 0.7,
         'colsample_bytree': 0.7,
         'seed': 0}
num_rounds = 150
month_train = '2015-06-28'
month_val = '2016-05-28'
weights = (4, 5, 6, 7, 8, 9)
weights = (1, 9, 10, 11, 12, 13, 14, 15)

history, model_dict = cv_month(param, num_rounds, month_train, month_val, n_repeat=7, random_seed=240,
                    lag_train=5, lag_val=5, weight_set=weights, verbose_eval=True)

Start weight index 1
##################################################
Train with weight 1, repetition 0 of 7
[0]	train-mlogloss:2.67584	val-mlogloss:2.71465	train-MAP@7:0.886399	val-MAP@7:0.839224
[1]	train-mlogloss:2.48599	val-mlogloss:2.54307	train-MAP@7:0.890687	val-MAP@7:0.841208
[2]	train-mlogloss:2.33731	val-mlogloss:2.40736	train-MAP@7:0.893118	val-MAP@7:0.846664
[3]	train-mlogloss:2.21383	val-mlogloss:2.29893	train-MAP@7:0.893906	val-MAP@7:0.847703
[4]	train-mlogloss:2.10816	val-mlogloss:2.20466	train-MAP@7:0.894769	val-MAP@7:0.848659
[5]	train-mlogloss:2.01616	val-mlogloss:2.12252	train-MAP@7:0.895373	val-MAP@7:0.848505
[6]	train-mlogloss:1.93473	val-mlogloss:2.04929	train-MAP@7:0.895746	val-MAP@7:0.849908
[7]	train-mlogloss:1.8615	val-mlogloss:1.98639	train-MAP@7:0.896417	val-MAP@7:0.849835
[8]	train-mlogloss:1.79538	val-mlogloss:1.93189	train-MAP@7:0.897094	val-MAP@7:0.849541
[9]	train-mlogloss:1.73474	val-mlogloss:1.88485	train-MAP@7:0.897791	val-MAP@7:0.849731
[10]	train

[91]	train-mlogloss:0.712588	val-mlogloss:1.02806	train-MAP@7:0.918396	val-MAP@7:0.85609
[92]	train-mlogloss:0.710517	val-mlogloss:1.02708	train-MAP@7:0.918483	val-MAP@7:0.855903
[93]	train-mlogloss:0.708329	val-mlogloss:1.02608	train-MAP@7:0.918693	val-MAP@7:0.855827
[94]	train-mlogloss:0.70627	val-mlogloss:1.02477	train-MAP@7:0.918804	val-MAP@7:0.855949
[95]	train-mlogloss:0.704251	val-mlogloss:1.02319	train-MAP@7:0.918909	val-MAP@7:0.856024
[96]	train-mlogloss:0.702335	val-mlogloss:1.02207	train-MAP@7:0.919123	val-MAP@7:0.856154
[97]	train-mlogloss:0.700511	val-mlogloss:1.02086	train-MAP@7:0.919256	val-MAP@7:0.856152
[98]	train-mlogloss:0.698685	val-mlogloss:1.01987	train-MAP@7:0.91938	val-MAP@7:0.856223
[99]	train-mlogloss:0.69672	val-mlogloss:1.01928	train-MAP@7:0.919583	val-MAP@7:0.856158
[100]	train-mlogloss:0.694978	val-mlogloss:1.0187	train-MAP@7:0.919652	val-MAP@7:0.855962
[101]	train-mlogloss:0.693316	val-mlogloss:1.01775	train-MAP@7:0.919685	val-MAP@7:0.855906
[102]	train-m

[31]	train-mlogloss:1.07309	val-mlogloss:1.33409	train-MAP@7:0.906128	val-MAP@7:0.85206
[32]	train-mlogloss:1.05786	val-mlogloss:1.32002	train-MAP@7:0.906474	val-MAP@7:0.852415
[33]	train-mlogloss:1.04327	val-mlogloss:1.30713	train-MAP@7:0.906611	val-MAP@7:0.851955
[34]	train-mlogloss:1.02937	val-mlogloss:1.29533	train-MAP@7:0.906745	val-MAP@7:0.85223
[35]	train-mlogloss:1.01628	val-mlogloss:1.28344	train-MAP@7:0.907255	val-MAP@7:0.852303
[36]	train-mlogloss:1.00348	val-mlogloss:1.27238	train-MAP@7:0.907486	val-MAP@7:0.852349
[37]	train-mlogloss:0.991491	val-mlogloss:1.26141	train-MAP@7:0.907735	val-MAP@7:0.852499
[38]	train-mlogloss:0.979935	val-mlogloss:1.25106	train-MAP@7:0.907861	val-MAP@7:0.852898
[39]	train-mlogloss:0.968849	val-mlogloss:1.24206	train-MAP@7:0.908202	val-MAP@7:0.85267
[40]	train-mlogloss:0.958326	val-mlogloss:1.23428	train-MAP@7:0.908385	val-MAP@7:0.852755
[41]	train-mlogloss:0.948048	val-mlogloss:1.2258	train-MAP@7:0.908694	val-MAP@7:0.85291
[42]	train-mlogloss:0

[123]	train-mlogloss:0.660548	val-mlogloss:1.00457	train-MAP@7:0.922928	val-MAP@7:0.855276
[124]	train-mlogloss:0.659326	val-mlogloss:1.00399	train-MAP@7:0.923182	val-MAP@7:0.855385
[125]	train-mlogloss:0.658111	val-mlogloss:1.00333	train-MAP@7:0.923326	val-MAP@7:0.855397
[126]	train-mlogloss:0.657002	val-mlogloss:1.00228	train-MAP@7:0.923452	val-MAP@7:0.855414
[127]	train-mlogloss:0.655681	val-mlogloss:1.00198	train-MAP@7:0.923621	val-MAP@7:0.855278
[128]	train-mlogloss:0.654666	val-mlogloss:1.00205	train-MAP@7:0.923758	val-MAP@7:0.855125
[129]	train-mlogloss:0.65366	val-mlogloss:1.00162	train-MAP@7:0.923842	val-MAP@7:0.855238
[130]	train-mlogloss:0.652462	val-mlogloss:1.00124	train-MAP@7:0.924058	val-MAP@7:0.855197
[131]	train-mlogloss:0.651361	val-mlogloss:1.00131	train-MAP@7:0.92417	val-MAP@7:0.855111
[132]	train-mlogloss:0.650178	val-mlogloss:1.00092	train-MAP@7:0.924299	val-MAP@7:0.855047
[133]	train-mlogloss:0.649043	val-mlogloss:1.00029	train-MAP@7:0.92446	val-MAP@7:0.855039
[1

[63]	train-mlogloss:0.79801	val-mlogloss:1.08851	train-MAP@7:0.913158	val-MAP@7:0.857595
[64]	train-mlogloss:0.793678	val-mlogloss:1.08543	train-MAP@7:0.913373	val-MAP@7:0.857942
[65]	train-mlogloss:0.789467	val-mlogloss:1.0825	train-MAP@7:0.913648	val-MAP@7:0.857782
[66]	train-mlogloss:0.785448	val-mlogloss:1.07843	train-MAP@7:0.913766	val-MAP@7:0.857816
[67]	train-mlogloss:0.781489	val-mlogloss:1.07592	train-MAP@7:0.913988	val-MAP@7:0.857839
[68]	train-mlogloss:0.777564	val-mlogloss:1.0728	train-MAP@7:0.914245	val-MAP@7:0.857802
[69]	train-mlogloss:0.773753	val-mlogloss:1.06988	train-MAP@7:0.914405	val-MAP@7:0.857946
[70]	train-mlogloss:0.770128	val-mlogloss:1.06725	train-MAP@7:0.914481	val-MAP@7:0.857955
[71]	train-mlogloss:0.766578	val-mlogloss:1.06487	train-MAP@7:0.914594	val-MAP@7:0.857936
[72]	train-mlogloss:0.763175	val-mlogloss:1.06186	train-MAP@7:0.914842	val-MAP@7:0.857863
[73]	train-mlogloss:0.759764	val-mlogloss:1.05933	train-MAP@7:0.914928	val-MAP@7:0.85785
[74]	train-mlo

...

[84]	train-mlogloss:0.804244	val-mlogloss:1.11133	train-MAP@7:0.917235	val-MAP@7:0.85787
[85]	train-mlogloss:0.801797	val-mlogloss:1.10936	train-MAP@7:0.917249	val-MAP@7:0.857954
[86]	train-mlogloss:0.799337	val-mlogloss:1.10806	train-MAP@7:0.917388	val-MAP@7:0.857702
[87]	train-mlogloss:0.797014	val-mlogloss:1.10592	train-MAP@7:0.917608	val-MAP@7:0.85765
[88]	train-mlogloss:0.794622	val-mlogloss:1.10395	train-MAP@7:0.917774	val-MAP@7:0.857781
[89]	train-mlogloss:0.792412	val-mlogloss:1.1022	train-MAP@7:0.91784	val-MAP@7:0.857599
[90]	train-mlogloss:0.790081	val-mlogloss:1.10053	train-MAP@7:0.918021	val-MAP@7:0.857683
[91]	train-mlogloss:0.787832	val-mlogloss:1.09872	train-MAP@7:0.918107	val-MAP@7:0.857799
[92]	train-mlogloss:0.785709	val-mlogloss:1.09697	train-MAP@7:0.91824	val-MAP@7:0.857977
[93]	train-mlogloss:0.783626	val-mlogloss:1.09625	train-MAP@7:0.918376	val-MAP@7:0.857898
[94]	train-mlogloss:0.781508	val-mlogloss:1.09519	train-MAP@7:0.918543	val-MAP@7:0.857524
[95]	train-mlog

[24]	train-mlogloss:1.27559	val-mlogloss:1.51208	train-MAP@7:0.904483	val-MAP@7:0.853517
[25]	train-mlogloss:1.25459	val-mlogloss:1.49444	train-MAP@7:0.904637	val-MAP@7:0.853714
[26]	train-mlogloss:1.23449	val-mlogloss:1.47959	train-MAP@7:0.904927	val-MAP@7:0.853683
[27]	train-mlogloss:1.21559	val-mlogloss:1.46428	train-MAP@7:0.905372	val-MAP@7:0.853717
[28]	train-mlogloss:1.19752	val-mlogloss:1.44744	train-MAP@7:0.905626	val-MAP@7:0.854136
[29]	train-mlogloss:1.18006	val-mlogloss:1.43348	train-MAP@7:0.906042	val-MAP@7:0.853909
[30]	train-mlogloss:1.16356	val-mlogloss:1.41808	train-MAP@7:0.906209	val-MAP@7:0.854034
[31]	train-mlogloss:1.14794	val-mlogloss:1.40536	train-MAP@7:0.906448	val-MAP@7:0.85389
[32]	train-mlogloss:1.133	val-mlogloss:1.39287	train-MAP@7:0.906888	val-MAP@7:0.853607
[33]	train-mlogloss:1.11863	val-mlogloss:1.38042	train-MAP@7:0.907244	val-MAP@7:0.853745
[34]	train-mlogloss:1.10499	val-mlogloss:1.371	train-MAP@7:0.907542	val-MAP@7:0.853685
[35]	train-mlogloss:1.0918

[116]	train-mlogloss:0.744938	val-mlogloss:1.07737	train-MAP@7:0.921694	val-MAP@7:0.854461
[117]	train-mlogloss:0.743632	val-mlogloss:1.0764	train-MAP@7:0.921811	val-MAP@7:0.854528
[118]	train-mlogloss:0.742169	val-mlogloss:1.07583	train-MAP@7:0.921981	val-MAP@7:0.854612
[119]	train-mlogloss:0.740831	val-mlogloss:1.0752	train-MAP@7:0.922078	val-MAP@7:0.854649
[120]	train-mlogloss:0.739451	val-mlogloss:1.0749	train-MAP@7:0.922261	val-MAP@7:0.854419
[121]	train-mlogloss:0.737909	val-mlogloss:1.07481	train-MAP@7:0.92253	val-MAP@7:0.854093
[122]	train-mlogloss:0.736623	val-mlogloss:1.07478	train-MAP@7:0.922607	val-MAP@7:0.85416
[123]	train-mlogloss:0.735184	val-mlogloss:1.07376	train-MAP@7:0.922824	val-MAP@7:0.854213
[124]	train-mlogloss:0.733851	val-mlogloss:1.07306	train-MAP@7:0.922963	val-MAP@7:0.854364
[125]	train-mlogloss:0.732568	val-mlogloss:1.07259	train-MAP@7:0.923093	val-MAP@7:0.854322
[126]	train-mlogloss:0.731379	val-mlogloss:1.0719	train-MAP@7:0.923261	val-MAP@7:0.854303
[127]

[56]	train-mlogloss:0.921674	val-mlogloss:1.206	train-MAP@7:0.912508	val-MAP@7:0.855829
[57]	train-mlogloss:0.916241	val-mlogloss:1.20114	train-MAP@7:0.912807	val-MAP@7:0.855896
[58]	train-mlogloss:0.91081	val-mlogloss:1.19662	train-MAP@7:0.912956	val-MAP@7:0.855903
[59]	train-mlogloss:0.905548	val-mlogloss:1.19239	train-MAP@7:0.91325	val-MAP@7:0.855915
[60]	train-mlogloss:0.900584	val-mlogloss:1.18826	train-MAP@7:0.913412	val-MAP@7:0.85608
[61]	train-mlogloss:0.895562	val-mlogloss:1.18398	train-MAP@7:0.913562	val-MAP@7:0.856185
[62]	train-mlogloss:0.890812	val-mlogloss:1.18013	train-MAP@7:0.913782	val-MAP@7:0.856137
[63]	train-mlogloss:0.88616	val-mlogloss:1.17637	train-MAP@7:0.913999	val-MAP@7:0.856347
[64]	train-mlogloss:0.881574	val-mlogloss:1.1731	train-MAP@7:0.914279	val-MAP@7:0.856262
[65]	train-mlogloss:0.877198	val-mlogloss:1.16925	train-MAP@7:0.914388	val-MAP@7:0.856341
[66]	train-mlogloss:0.8729	val-mlogloss:1.16596	train-MAP@7:0.914566	val-MAP@7:0.856626
[67]	train-mlogloss

[148]	train-mlogloss:0.71837	val-mlogloss:1.07737	train-MAP@7:0.926851	val-MAP@7:0.854093
[149]	train-mlogloss:0.717464	val-mlogloss:1.07712	train-MAP@7:0.927149	val-MAP@7:0.854014
Validate logloss = 1.07712, MAP@7 = 0.85401, time = 13.33 min
--------------------------------------------------

Train with weight 14, repetition 1 of 7
[0]	train-mlogloss:2.69878	val-mlogloss:2.72732	train-MAP@7:0.883803	val-MAP@7:0.843106
[1]	train-mlogloss:2.52139	val-mlogloss:2.57139	train-MAP@7:0.889208	val-MAP@7:0.847338
[2]	train-mlogloss:2.38038	val-mlogloss:2.44723	train-MAP@7:0.892405	val-MAP@7:0.848285
[3]	train-mlogloss:2.26264	val-mlogloss:2.34158	train-MAP@7:0.893698	val-MAP@7:0.849422
[4]	train-mlogloss:2.16192	val-mlogloss:2.25303	train-MAP@7:0.894926	val-MAP@7:0.85
[5]	train-mlogloss:2.07371	val-mlogloss:2.17595	train-MAP@7:0.895657	val-MAP@7:0.851856
[6]	train-mlogloss:1.99526	val-mlogloss:2.11014	train-MAP@7:0.896272	val-MAP@7:0.853233
[7]	train-mlogloss:1.92484	val-mlogloss:2.04745	train

In [ ]:
history_prev = pd.read_csv('weight_cv_2_history.csv', header=[0, 1, 2, 3], index_col=0)
history_prev.columns.set_levels(history_prev.columns.levels[0].astype(int), level=0, inplace=True) # convert string to int
history_prev.columns.set_levels(history_prev.columns.levels[1].astype(int), level=1, inplace=True) # convert string to int
val = {}

# load previous result w=exp(1/n-1)
val[1, 'mean'] = history_prev.xs(axis=1, level=[0, 2, 3], key=(9, 'val', 'MAP@7')).apply('mean', axis=1)
val[1, 'std'] = history_prev.xs(axis=1, level=[0, 2, 3], key=(9, 'val', 'MAP@7')).apply('std', axis=1)

# process current results
for idx, metric in itertools.product(weights, ['mean', 'std']):
    val[(idx, metric)] = history.xs(axis=1, level=[0, 2, 3], key=(idx, 'val', 'MAP@7')).apply(metric, axis=1)

In [ ]:
plt.figure(figsize=(16, 9))

plt.plot(val[1, 'mean'], label='weight=exp(1/n-1)')
plt.fill_between(history_prev.index, val[1, 'mean']-val[1, 'std'], val[1, 'mean']+val[1, 'std'], alpha=0.3)

nd = {1: 1, 9: 0.7, 10: 0.6, 11: 0.5, 12: 0.4, 13: 0.3, 14: 0.2, 15: 0.1}
for idx in weights:
    plt.plot(val[idx, 'mean'], label='weight=exp({}/n-{})'.format(nd[idx], nd[idx]))
    plt.fill_between(history.index, val[idx, 'mean']-val[idx, 'std'], val[idx, 'mean']+val[idx, 'std'], alpha=0.3)

plt.grid()
# plt.xlim(50, 100)
# plt.ylim(0.85, 0.86)
plt.legend()

In [ ]:
history.to_csv('weight_cv_3_history.csv')

In [ ]:
save_pickle('weight_cv_3_models.pickle', model_dict)

In [ ]:
history